In [ ]:
%install '.package(path: "$cwd/FastaiNotebook_08a_heterogeneous_dictionary")' FastaiNotebook_08a_heterogeneous_dictionary

Installing packages:
	.package(path: "/home/ubuntu/fastai_docs/dev_swift/FastaiNotebook_08a_heterogeneous_dictionary")
		FastaiNotebook_08a_heterogeneous_dictionary
With SwiftPM flags: []
Working in: /tmp/tmprf0ly84y/swift-install
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 4.43s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'FastaiNotebook_08a_heterogeneous_dictionary' (13 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


## Load data

In [ ]:
import FastaiNotebook_08a_heterogeneous_dictionary
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


In [ ]:
// export
import Path
import TensorFlow

In [ ]:
let path = downloadImagette()

In [ ]:
let il = ItemList(fromFolder: path, extensions: ["jpeg", "jpg"])
let sd = SplitData(il, fromFunc: {grandParentSplitter(fName: $0, valid: "val")})
var (procItem,procLabel) = (NoopProcessor<Path>(),CategoryProcessor())
let sld = SplitLabeledData(sd, fromFunc: parentLabeler, procItem: &procItem, procLabel: &procLabel)
var rawData = sld.toDataBunch(itemToTensor: pathsToTensor, labelToTensor: intsToTensor)
let data = transformData(rawData, tfmItem: { openAndResize(fname: $0, size: 128) })

In [ ]:
let data = mnistDataBunch(flat: true)

In [ ]:
let (n,m) = (60000,784)
let c = 10
let nHid = 50

In [ ]:
func modelInit() -> BasicModel {return BasicModel(nIn: m, nHid: nHid, nOut: c)}

## Stateful optimizer

In [ ]:
//export
open class StatDelegate<Scalar: TensorFlowFloatingPoint> {
    open var name: String { return "" }
    var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary() }
    func update(
        state: inout [String: Tensor<Scalar>],
        for param: Tensor<Scalar>,
        along direction: Tensor<Scalar>,
        config: inout HeterogeneousDictionary
    ) { }
}

//export
open class StepDelegate<Scalar: TensorFlowFloatingPoint> {
    var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary() }
    func update(
        param: inout Tensor<Scalar>,
        along direction: inout Tensor<Scalar>,
        state: [String: Tensor<Scalar>],
        config: inout HeterogeneousDictionary
    ) { }
}

In [ ]:
//export
class StatefulOptimizer<Model: Layer,
                        Scalar: TensorFlowFloatingPoint>: Optimizer
    where Model.AllDifferentiableVariables == Model.CotangentVector{
    var configs: [HeterogeneousDictionary]
    var learningRate: Float {
        get { return configs.last![LearningRate()] } 
        set { 
            for i in configs.indices {self.configs[i][LearningRate()] = newValue }
        }
    }
    var learningRates: [Float] {
        get {
            var res: [Float] = []
            for config in configs {res.append(config[LearningRate()])}
            return res
        }
        set { 
            for i in configs.indices {self.configs[i][LearningRate()] = newValue[i] } 
        }
    }
    var splits: (Int) -> Int
    var states: [String: Model.AllDifferentiableVariables]
    var statDelegates: [StatDelegate<Scalar>]
    var stepDelegates: [StepDelegate<Scalar>]
    init(
        stepDelegates: [StepDelegate<Scalar>],
        statDelegates: [StatDelegate<Scalar>],
        configs: [HeterogeneousDictionary],
        splits: @escaping (Int) -> Int
    ) {
        self.configs = Array(repeating: HeterogeneousDictionary(), count: configs.count)
        states = [:]
        for stepDelegate in stepDelegates {
            for i in self.configs.indices { self.configs[i].merge(stepDelegate.defaultConfig) { (_, new) in new } }
        }
        for statDelegate in statDelegates {
            for i in self.configs.indices { self.configs[i].merge(statDelegate.defaultConfig) { (_, new) in new } }
            states[statDelegate.name] = Model.AllDifferentiableVariables.zero
        }
        for i in 0..<configs.count {
            self.configs[i].merge(configs[i]) { (_, new) in new }
        }
        self.stepDelegates = stepDelegates
        self.statDelegates = statDelegates
        self.splits = splits
    }
        
    func update(
        _ model: inout Model.AllDifferentiableVariables,
        along direction: Model.CotangentVector
    ) {
        for (i,kp) in model.recursivelyAllWritableKeyPaths(to: Tensor<Scalar>.self).enumerated() {
            var grad = direction[keyPath: kp]
            var state = states.mapValues(){$0[keyPath: kp]}
            var config = configs[splits(i)]
            for statDelegate in statDelegates {
                statDelegate.update(
                    state: &state,
                    for: model[keyPath: kp],
                    along: grad,
                    config: &config
                )
            }
            for n in states.keys { states[n]![keyPath: kp] = state[n]! }
            for stepDelegate in stepDelegates {
                stepDelegate.update(
                    param: &model[keyPath: kp],
                    along: &grad,
                    state: state,
                    config: &config
                )
            }
        }
    }
}

In [ ]:
//export
class SGDStep: StepDelegate<Float> {
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= direction * config[LearningRate()]
    }
}

In [ ]:
//export
public struct WeightDecayKey: HetDictKey, Equatable {
    public static var defaultValue: Float = 0.0
}

class WeightDecay: StepDelegate<Float> {
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param *= 1 - config[LearningRate()] * config[WeightDecayKey()]
    }
}

In [ ]:
//export

class L2Regularization: StepDelegate<Float> {
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        direction += config[WeightDecayKey()] * param
    }
}

In [ ]:
//export

public struct Momentum: HetDictKey, Equatable {
    public static var defaultValue: Float = 0.9
}

public struct MomentumDampening: HetDictKey, Equatable {
    public static var defaultValue: Float = 0.9
}

class AverageGrad: StatDelegate<Float> {
    let dampened: Bool
    init(dampened: Bool = false) { self.dampened = dampened }
    override var name: String { return "averageGrad" }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state["averageGrad"]! *= config[Momentum()]
        config[MomentumDampening()] = 1.0 - (dampened ? config[Momentum()] : 0.0)
        state["averageGrad"]! += config[MomentumDampening()] * direction
    }
}

In [ ]:
func splitFunc(_ a: Int) -> Int { return a < 2 ? 0 : 1 }

In [ ]:
var configs = [HeterogeneousDictionary(LearningRate(), 0.0), HeterogeneousDictionary(LearningRate(), 0.01)]
func optFunc(_ model: BasicModel) -> StatefulOptimizer<BasicModel, Float> {
    return StatefulOptimizer(
        stepDelegates: [SGDStep()],
        statDelegates: [],
        configs: configs,
        splits: splitFunc)
}

In [ ]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
let params = learner.model.allDifferentiableVariables

In [ ]:
for kp in params.recursivelyAllWritableKeyPaths(to: TF.self) { 
    print(params[keyPath: kp][0]) 
}

[  0.009547793,    -0.0652047,  -0.040830057,   -0.06607734,  -0.035449643,  -0.029223876,
  -0.069044866,   -0.07417027,  -0.044368744,  -0.017305592,  -0.054592714,   -0.07292264,
   0.004027301,   -0.05478491,   -0.08444236,   -0.01053853,    0.08308772,    0.05292193,
 -0.0041120634,  -0.025117364,  -0.041467123,   -0.06437787,   0.018266529,   0.007423208,
   -0.05540281,  -0.054200035,  -0.062669456,   0.010964707,   0.016433652,  -0.028503189,
   -0.05840129,    0.07455959,  -0.080125235,   0.012823347,  0.0046170265,  -0.060755327,
    -0.0266574,   0.064538606,  -0.010068561,   0.021664713,   -0.06170307,    -0.0270647,
   -0.04225708,   -0.05714935, -0.0010171867,    0.08358967,    0.04058204,  -0.029697666,
  0.0042612036,    0.08436677]
0.0
[ -0.076215886, 0.00037851845,   -0.19870798,   -0.10219983,   -0.21089736,    0.08636105,
    0.20502812,    0.14080973,   -0.14035013,    0.24389455]
0.0


In [ ]:
learner.fit(2)

Epoch 0: [1.1411686, 0.6516]                                                    
Epoch 1: [0.9434332, 0.7135]                                                    


In [ ]:
let params = learner.model.allDifferentiableVariables
for kp in params.recursivelyAllWritableKeyPaths(to: TF.self) { 
    print(params[keyPath: kp][0]) 
}

[  0.009547793,    -0.0652047,  -0.040830057,   -0.06607734,  -0.035449643,  -0.029223876,
  -0.069044866,   -0.07417027,  -0.044368744,  -0.017305592,  -0.054592714,   -0.07292264,
   0.004027301,   -0.05478491,   -0.08444236,   -0.01053853,    0.08308772,    0.05292193,
 -0.0041120634,  -0.025117364,  -0.041467123,   -0.06437787,   0.018266529,   0.007423208,
   -0.05540281,  -0.054200035,  -0.062669456,   0.010964707,   0.016433652,  -0.028503189,
   -0.05840129,    0.07455959,  -0.080125235,   0.012823347,  0.0046170265,  -0.060755327,
    -0.0266574,   0.064538606,  -0.010068561,   0.021664713,   -0.06170307,    -0.0270647,
   -0.04225708,   -0.05714935, -0.0010171867,    0.08358967,    0.04058204,  -0.029697666,
  0.0042612036,    0.08436677]
0.0
[  0.11427569,   -0.2975522,  -0.27106583,  -0.22690077,   -0.2002997,   0.17686953,
    0.6566362, -0.039751936, -0.116897374,   0.15278816]
-0.04700609


In [ ]:
//export
class MomentumStep: StepDelegate<Float> {
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= config[LearningRate()] * state["averageGrad"]!
    }
}

In [ ]:
func optFunc(_ model: BasicModel) -> StatefulOptimizer<BasicModel, Float> {
    return StatefulOptimizer(
        stepDelegates: [MomentumStep()],
        statDelegates: [AverageGrad()],
        configs: configs,
        splits: splitFunc)
}

In [ ]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
learner.fit(2)

Epoch 0: [0.6913208, 0.7908]                                                    
Epoch 1: [0.6574781, 0.7991]                                                    


In [ ]:
//export

public struct SquareMomentum: HetDictKey, Equatable {
    public static var defaultValue: Float = 0.99
}

public struct SquareMomentumDampening: HetDictKey, Equatable {
    public static var defaultValue: Float = 0.99
}


class AverageSquaredGrad: StatDelegate<Float> {
    let dampened: Bool
    init(dampened: Bool = false) { self.dampened = dampened }
    override var name: String { return "averageSquaredGrad" }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state["averageSquaredGrad"]! *= config[SquareMomentum()]
        config[SquareMomentumDampening()] = 1.0 - (dampened ? config[SquareMomentum()] : 0.0)
        state["averageSquaredGrad"]! += config[SquareMomentumDampening()] * direction.squared()
    }
}

In [ ]:
//export
class StepCount: StatDelegate<Float> {
    override var name: String { return "step" }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state["step"]! += 1.0
    }
}

In [ ]:
//export
func debias<Scalar: TensorFlowFloatingPoint>(_ mom: Scalar, _ damp: Scalar, _ step: Tensor<Scalar>) -> Tensor<Scalar> {
    return damp * (1 - pow(mom, step)) / (1 - mom)
}

In [ ]:
//export
public struct Epsilon: HetDictKey, Equatable {
    public static var defaultValue: Float = 1e-5
}

class AdamStep: StepDelegate<Float> {
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        let debiasedLearningRate = config[LearningRate()] / debias(
            config[Momentum()], config[MomentumDampening()], state["step"]!)
        let debiasedRMSGrad = sqrt(state["averageSquaredGrad"]! / debias(
            config[SquareMomentum()], config[SquareMomentumDampening()], state["step"]!)) + config[Epsilon()]
        param -= debiasedLearningRate * state["averageGrad"]! / debiasedRMSGrad
    }
}

In [ ]:
func opt(_ model: BasicModel) -> StatefulOptimizer<BasicModel, Float> {
    return StatefulOptimizer(
        stepDelegates: [AdamStep()], 
        statDelegates: [AverageGrad(), AverageSquaredGrad(), StepCount()], 
        configs: configs,
        splits: splitFunc)
}

In [ ]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
learner.fit(2)

Epoch 0: [0.7322381, 0.7717]                                                    
Epoch 1: [0.6920017, 0.7842]                                                    


In [ ]:
class LambStep: StepDelegate<Float> {
    override var defaultConfig: HeterogeneousDictionary {
        return HeterogeneousDictionary(Epsilon(), 1e-6, WeightDecayKey(), 0.0)
    }
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        let debiasedAverageGrad = state["averageGrad"]! / debias(
            momentum: config[Momentum()],
            dampening: config[MomentumDampening()],
            step: state["step"]!
        )
        let debiasedRMSGrad = sqrt(state["averageSquaredGrad"]! / debias(
            momentum: config[SquareMomentum()],
            dampening: config[SquareMomentumDampening()],
            step: state["step"]!
        ) + config[Epsilon()])
        let step = debiasedAverageGrad / debiasedRMSGrad + config[WeightDecayKey()] * param
        let r1 = sqrt((param * param).mean())
        let r2 = sqrt((step * step).mean())
        let factor = min(r1 / r2, Float(10.0))
        param -= config[LearningRate()] * factor * step
    }
}

## Export

In [ ]:
notebookToScript(fname: (Path.cwd / "09_optimizer.ipynb").string)